In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import string
import re
# import emoji
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [38]:
# !pip install nltk

In [39]:
df = pd.read_csv('./dataset/polisi_kanjuruhan_suporter_08-05-2023_06-11-21.csv', on_bad_lines='skip', sep=';')
df.head()

,created_at,id,id_str,full_text,quote_count,reply_count,retweet_count,favorite_count,geo,lang,user_id_str,conversation_id,conversation_id_str,media_url_https,media_type,username
0,Fri Oct 07 12:41:45 +0000 2022,1578364957166833700,1578364957166833664,“Lebih dari seratus orang dibunuh oleh polisi....,0,1,3,7,NaN,in,569265633,1578364957166833700,1578364957166833664,NaN,NaN,islamidotco
1,Fri Oct 07 18:26:56 +0000 2022,1578451828714672000,1578451828714672128,"Kalo harus cepet, klub bisa manuver minta pert...",0,1,0,0,NaN,in,1572482866097041408,1578451795609026600,1578451795609026560,NaN,NaN,_kang_bakso_
2,Fri Oct 07 16:19:46 +0000 2022,1578419822735392800,1578419822735392771,FIFA tidak berikan SANKSI kepada SEPAKBOLA IND...,0,0,0,1,NaN,in,1203075185420599296,1578419822735392800,1578419822735392771,NaN,NaN,Budi_2232
3,Fri Oct 07 07:41:49 +0000 2022,1578289477046239200,1578289477046239232,"Kisah Iwan, Aremania yang Meninggal saat Menco...",0,0,1,1,NaN,in,58880688,1578289477046239200,1578289477046239232,NaN,NaN,pikiran_rakyat
4,Fri Oct 07 07:30:05 +0000 2022,1578286526035021800,1578286526035021824,Polisi menembakkan gas air mata saat menghalau...,0,0,0,0,NaN,in,902837643024539651,1578286526035021800,1578286526035021824,https://pbs.twimg.com/media/FeczNlpaMAA_otM.jpg,photo,medcom_id


In [40]:
df.shape

(1065, 16)

In [41]:
df['full_text']

0       “Lebih dari seratus orang dibunuh oleh polisi....
1       Kalo harus cepet, klub bisa manuver minta pert...
2       FIFA tidak berikan SANKSI kepada SEPAKBOLA IND...
3       Kisah Iwan, Aremania yang Meninggal saat Menco...
4       Polisi menembakkan gas air mata saat menghalau...
                              ...                        
1060    Innalillahi..\n\nSpeechless.\n\nhttps://t.co/D...
1061    Alasan Polisi Tembakkan Gas Air Mata saat Keru...
1062    127 supporter termasuk anak2 meninggal di stad...
1063    Pihak yg jadi penyebab tragedi #Kanjuruhan :\n...
1064    Ini akibat ketololan oknum suporter saja , yg ...
Name: full_text, Length: 1065, dtype: object

In [42]:
data = df[['created_at', 'full_text']]

In [43]:
data

,created_at,full_text
0,Fri Oct 07 12:41:45 +0000 2022,“Lebih dari seratus orang dibunuh oleh polisi....
1,Fri Oct 07 18:26:56 +0000 2022,"Kalo harus cepet, klub bisa manuver minta pert..."
2,Fri Oct 07 16:19:46 +0000 2022,FIFA tidak berikan SANKSI kepada SEPAKBOLA IND...
3,Fri Oct 07 07:41:49 +0000 2022,"Kisah Iwan, Aremania yang Meninggal saat Menco..."
4,Fri Oct 07 07:30:05 +0000 2022,Polisi menembakkan gas air mata saat menghalau...
...,...,...
1060,Sun Oct 02 00:09:23 +0000 2022,Innalillahi..\n\nSpeechless.\n\nhttps://t.co/D...
1061,Sun Oct 02 00:07:07 +0000 2022,Alasan Polisi Tembakkan Gas Air Mata saat Keru...
1062,Sun Oct 02 00:07:59 +0000 2022,127 supporter termasuk anak2 meninggal di stad...
1063,Sun Oct 02 00:07:22 +0000 2022,Pihak yg jadi penyebab tragedi #Kanjuruhan :\n...


In [44]:
data.loc[1061][1]

'Alasan Polisi Tembakkan Gas Air Mata saat Kerusuhan Suporter di Stadion Kanjuruhan Malang https://t.co/py05pcV2BZ'

In [45]:
data.duplicated().sum()

4

In [46]:
data.drop_duplicates(inplace=True)

/tmp/ipykernel_13396/2282015914.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop_duplicates(inplace=True)


In [47]:
data.duplicated().sum()

0

In [48]:
def clean_word(tweet):
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet = re.sub(r'#', '', tweet)
    tweet = re.sub(r'https\S+', '', tweet)
    tweet = re.sub(r'[0-9]+', '', tweet)
    tweet = re.sub(r"\_", " ", tweet)
    tweet = re.sub("\W+", " ", tweet)
    tweet = re.sub(r'\\n', " ", tweet)
    tweet = re.sub(r':', '', tweet)
    
    return tweet

In [49]:
data.head(2)

,created_at,full_text
0,Fri Oct 07 12:41:45 +0000 2022,“Lebih dari seratus orang dibunuh oleh polisi....
1,Fri Oct 07 18:26:56 +0000 2022,"Kalo harus cepet, klub bisa manuver minta pert..."


In [50]:
data['clean_text'] = data['full_text'].apply(lambda x: clean_word(x))

/tmp/ipykernel_13396/3397790900.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['clean_text'] = data['full_text'].apply(lambda x: clean_word(x))


In [51]:
data.head()

,created_at,full_text,clean_text
0,Fri Oct 07 12:41:45 +0000 2022,“Lebih dari seratus orang dibunuh oleh polisi....,Lebih dari seratus orang dibunuh oleh polisi ...
1,Fri Oct 07 18:26:56 +0000 2022,"Kalo harus cepet, klub bisa manuver minta pert...",Kalo harus cepet klub bisa manuver minta perta...
2,Fri Oct 07 16:19:46 +0000 2022,FIFA tidak berikan SANKSI kepada SEPAKBOLA IND...,FIFA tidak berikan SANKSI kepada SEPAKBOLA IND...
3,Fri Oct 07 07:41:49 +0000 2022,"Kisah Iwan, Aremania yang Meninggal saat Menco...",Kisah Iwan Aremania yang Meninggal saat Mencob...
4,Fri Oct 07 07:30:05 +0000 2022,Polisi menembakkan gas air mata saat menghalau...,Polisi menembakkan gas air mata saat menghalau...


In [52]:
data['clean_text'] = data['clean_text'].str.lower()

/tmp/ipykernel_13396/584121520.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['clean_text'] = data['clean_text'].str.lower()


In [53]:
data.head(2)

,created_at,full_text,clean_text
0,Fri Oct 07 12:41:45 +0000 2022,“Lebih dari seratus orang dibunuh oleh polisi....,lebih dari seratus orang dibunuh oleh polisi ...
1,Fri Oct 07 18:26:56 +0000 2022,"Kalo harus cepet, klub bisa manuver minta pert...",kalo harus cepet klub bisa manuver minta perta...


In [54]:
data['clean_text'][:10]

0     lebih dari seratus orang dibunuh oleh polisi ...
1    kalo harus cepet klub bisa manuver minta perta...
2    fifa tidak berikan sanksi kepada sepakbola ind...
3    kisah iwan aremania yang meninggal saat mencob...
4    polisi menembakkan gas air mata saat menghalau...
5    tragedi kanjuruhan terulang di argentina supor...
6    polisi kantongi identitas suporter perusak mob...
7    kapolri jenderal polisi listyo sigit prabowo m...
8    suporter bayern muchen mengenang kanjuruhan nl...
9    saya telah menerima surat dari fifa menyusul p...
Name: clean_text, dtype: object

In [55]:
data['clean_text'] = data['clean_text'].str.replace("[^a-zA-Z#]", " ")

/tmp/ipykernel_13396/1107246706.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['clean_text'] = data['clean_text'].str.replace("[^a-zA-Z#]", " ")


In [56]:
data

,created_at,full_text,clean_text
0,Fri Oct 07 12:41:45 +0000 2022,“Lebih dari seratus orang dibunuh oleh polisi....,lebih dari seratus orang dibunuh oleh polisi ...
1,Fri Oct 07 18:26:56 +0000 2022,"Kalo harus cepet, klub bisa manuver minta pert...",kalo harus cepet klub bisa manuver minta perta...
2,Fri Oct 07 16:19:46 +0000 2022,FIFA tidak berikan SANKSI kepada SEPAKBOLA IND...,fifa tidak berikan sanksi kepada sepakbola ind...
3,Fri Oct 07 07:41:49 +0000 2022,"Kisah Iwan, Aremania yang Meninggal saat Menco...",kisah iwan aremania yang meninggal saat mencob...
4,Fri Oct 07 07:30:05 +0000 2022,Polisi menembakkan gas air mata saat menghalau...,polisi menembakkan gas air mata saat menghalau...
...,...,...,...
1060,Sun Oct 02 00:09:23 +0000 2022,Innalillahi..\n\nSpeechless.\n\nhttps://t.co/D...,innalillahi n nspeechless n n orang menjadi ko...
1061,Sun Oct 02 00:07:07 +0000 2022,Alasan Polisi Tembakkan Gas Air Mata saat Keru...,alasan polisi tembakkan gas air mata saat keru...
1062,Sun Oct 02 00:07:59 +0000 2022,127 supporter termasuk anak2 meninggal di stad...,supporter termasuk anak meninggal di stadion ...
1063,Sun Oct 02 00:07:22 +0000 2022,Pihak yg jadi penyebab tragedi #Kanjuruhan :\n...,pihak yg jadi penyebab tragedi kanjuruhan n ps...


In [57]:
data['clean_text'].loc[1063]

'pihak yg jadi penyebab tragedi kanjuruhan n pssi dan pt lib karena menolak perubahan jadwal yang dimohonkan arema hanya karena kepentingan rating semata n polisi yang menggunakan gas air mata n penyebab utama suporter arema yg masuk ke lapangan dan memicu kericuhan '

In [58]:
# Remove short words
data['clean_text'] = data['clean_text'].apply(lambda x: " ".join([i for i in x.split() if len(i)>2]))

/tmp/ipykernel_13396/4231270860.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['clean_text'] = data['clean_text'].apply(lambda x: " ".join([i for i in x.split() if len(i)>2]))


In [59]:
data['clean_text'].loc[1063]

'pihak jadi penyebab tragedi kanjuruhan pssi dan lib karena menolak perubahan jadwal yang dimohonkan arema hanya karena kepentingan rating semata polisi yang menggunakan gas air mata penyebab utama suporter arema masuk lapangan dan memicu kericuhan'

In [60]:
from googletrans.client import Translator
from transformers import pipeline
from tensorflow.python.keras.saving import hdf5_format

In [61]:
# !pip install googletrans
# !pip install transformers

In [62]:
data.head(2)

,created_at,full_text,clean_text
0,Fri Oct 07 12:41:45 +0000 2022,“Lebih dari seratus orang dibunuh oleh polisi....,lebih dari seratus orang dibunuh oleh polisi k...
1,Fri Oct 07 18:26:56 +0000 2022,"Kalo harus cepet, klub bisa manuver minta pert...",kalo harus cepet klub bisa manuver minta perta...


In [63]:
data['clean_text'].isnull().sum()

0

In [64]:
translator = Translator()
sentiment_classifier = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [65]:
# !pip install keras
# !pip uninstall tensorflow==2.10 

In [66]:
data['label'] = data['clean_text'].str.encode('ascii', 'ignore').apply(translator.translate, dest='en')
data['label'] = data['label'].apply(getattr, args=('text',))

AttributeError: 'NoneType' object has no attribute 'group'

In [ ]:
data = ["I love you", "I hate you"]
sentiment_classifier(data)